In [3]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from datetime import datetime, timedelta, time
import heapq

In [4]:
file_path = "/Users/Apple/Documents/Data Engineering/Match Dump-DE Take home assignment.xlsx"
df = pd.read_excel(file_path)

print(df.head())

   Match No.       Date Time (IST)       Team A    Team B     Location  \
0          1 2024-08-15   14:00:00        India  Pakistan        India   
1          8 2024-08-15   14:00:00        India  Pakistan        India   
2         15 2024-08-15   18:00:00  West Indies   England  West Indies   
3         22 2024-08-15   14:00:00    Sri Lanka     India    Sri Lanka   
4         29 2024-08-15   14:00:00      England  Pakistan      England   

  Time Zone Diff for Opposition Gender    League/Event            Rivalry  \
0                           IST    Men       World Cup  India vs Pakistan   
1                           IST    Men       World Cup  India vs Pakistan   
2                 AST (IST-9.5)    Men  ENG tour of WI                NaN   
3                SLST (IST-0.5)    Men        Asia Cup                NaN   
4                 BST (IST-4.5)    Men       World Cup                NaN   

     Series Type  No. of Teams Match Type Match Category Finish Time (IST)  
0      World Cu

In [5]:
#current_timestamp = datetime.now()
current_timestamp = datetime(2024, 8, 16, 13, 0, 0)

def parse_time(time_str):
    return pd.to_datetime(time_str, format='%H:%M:%S').time()

def get_match_status(row, current_time):

    date = row['Date'].date()
    
    start_time = parse_time(row['Time (IST)'])
    finish_time = parse_time(row['Finish Time (IST)'])

    start_datetime = datetime.combine(date, start_time)       date = row['Date'].date()
    finish_datetime = datetime.combine(date, finish_time)
    
    if finish_datetime < start_datetime:
        finish_datetime += timedelta(days=1)
    
    if start_datetime <= current_time <= finish_datetime:
        return 'Live'
    elif current_time < start_datetime:
        return 'Upcoming'
    else:
        return 'Completed'

df['Status'] = df.apply(lambda row: get_match_status(row, current_timestamp), axis=1)

print(df[['Date', 'Time (IST)', 'Finish Time (IST)', 'Status']].head())


        Date Time (IST) Finish Time (IST)     Status
0 2024-08-15   14:00:00          22:00:00  Completed
1 2024-08-15   14:00:00          18:00:00  Completed
2 2024-08-15   18:00:00          22:00:00  Completed
3 2024-08-15   14:00:00          22:00:00  Completed
4 2024-08-15   14:00:00          18:00:00  Completed


In [5]:
output_file_path = "/Users/Apple/Documents/Data Engineering/Match Dump-DE Take home assignment.xlsx"
df.to_excel(output_file_path, index=False, engine='openpyxl')

print(f"Updated DataFrame saved to {output_file_path}")

Updated DataFrame saved to /Users/Apple/Documents/Data Engineering/Updated_Match_Dump.xlsx


In [29]:
def calculate_priority(row):
    priority = 0

    weights = {
        'teams': 0.01,             #0.095
        'time': 0.0057,            #0.0399
        'gender': 0.02166,         #0.0649
        'match_type': 0.02,        #0.06     
        'league': 0.01,            #0.1
        'num_teams': 0.0008333,    # 0.01
        'match_category': 0.13,    #0.13
        'status': 0.04,            #0.4
        'special_match': 0.03333   #0.1
    }
    
    teams_priority = {
        'India': 10, 'England': 9, 'Australia': 8, 'South Africa': 7, 'Pakistan': 6,
        'New Zealand': 5, 'Sri Lanka': 4, 'West Indies': 3, 'Afghanistan': 2, 'Others': 1
    }
    team_a_priority = teams_priority.get(row['Team A'], 10)
    team_b_priority = teams_priority.get(row['Team B'], 10)
    avg_team_priority = (team_a_priority + team_b_priority) / 2
    priority += avg_team_priority * weights['teams']
    print()
    

    time = row['Time (IST)']
    time_priority = {
        '17:00': 7, '12:00': 6, '20:30': 5, '09:00': 4, '23:00': 3, 
        '01:00': 2, '06:00': 1
    }
    
    time_str = time.strftime('%H:%M')
    

    time_intervals = sorted(time_priority.keys(), reverse=True)
    for interval in time_intervals:
        if time_str >= interval:
            priority += time_priority[interval] * weights['time']
            break
    

    gender_priority = 3 if row['Gender'] == 'Male' else 1
    priority += gender_priority * weights['gender']
    

    match_type_priority = {'T20I': 3, 'ODI': 2, 'Test': 1}
    match_type_value = match_type_priority.get(row['Match Type'], 4)
    priority += match_type_value * weights['match_type']
    

    known_leagues = {
        'World Cup':10 , 'IPL':9, 'BBL':6, 'CPL':6, 'Asia Cup':8, 'The Ashes':7, 'SA20': 4, 
        'Tri-Series': 3, 'European League': 2, 'ICC Women\'s T20 World Cup Qualifier Warm-up Matches': 1
    }
    league_priority = next((known_leagues[league] for league in known_leagues if league in row['League/Event']), 1)
    priority += league_priority * weights['league']
    

    num_teams_priority = row['No. of Teams']                          #max = 12
    priority += num_teams_priority * weights['num_teams']
    

    match_category_priority = 1 if row['Match Category'] == 'International' else 0.5
    priority += match_category_priority * weights['match_category']

    
    status_priority = {'Live': 10, 'Upcoming': 4, 'Completed': 1}
    status_value = status_priority.get(row['Status'], 4)
    priority += status_value * weights['status']

    
    if row['Match Type'] == 'Final':
        priority += 3 * weights['special_match']
    elif row['Match Type'] == 'Semi final':
        priority += 2 * weights['special_match']
    elif row['Match Type'] == 'Quarter':
        priority += 1 * weights['special_match']
    
    # Rivalry priority
    if (row['Team A'] == 'India' and row['Team B'] == 'Pakistan') or (row['Team A'] == 'Pakistan' and row['Team B'] == 'India'):
        priority += 0.1  

    if (row['Team A'] == 'England' and row['Team B'] == 'Australia') or (row['Team A'] == 'Australia' and row['Team B'] == 'England'):
        priority += 0.07  

    
    return priority


df['Priority'] = df.apply(calculate_priority, axis=1)


# completed match logic
completed_matches = df[df['Status'] == 'Completed']
if len(completed_matches) > 3:
    completed_matches = completed_matches.nlargest(3, 'Priority')
    df = pd.concat([df[df['Status'] != 'Completed'], completed_matches], ignore_index=True)


# (8 AM - 8 PM) logic
current_time = datetime(2024, 8, 16, 13, 0, 0).time()

df['Time (IST)'] = pd.to_datetime(df['Time (IST)'], format='%H:%M:%S').dt.time
start_time = time(8, 0)  
end_time = time(20, 0)  

if start_time <= current_time <= end_time:
    df = df[(df['Status'] != 'Completed') | (df['Time (IST)'].between(start_time, end_time))]


# 24 hr display logic
df['Date'] = pd.to_datetime(df['Date']) 
df['Match DateTime'] = df.apply(lambda row: datetime.combine(row['Date'].date(), row['Time (IST)']), axis=1)     

current_datetime = datetime(2024, 8, 16, 13, 0, 0)
df = df[(df['Match DateTime'] - current_datetime).dt.total_seconds() / 3600 <= 24]


priority_queue = []
for _, row in df.iterrows():
    heapq.heappush(priority_queue, (-row['Priority'], _, row.to_dict()))

prioritized_matches = []
while priority_queue:
    _, _, match = heapq.heappop(priority_queue)
    prioritized_matches.append(match)

df_sorted = pd.DataFrame(prioritized_matches)

output_columns = ['Match No.', 'Date', 'Time (IST)', 'Team A', 'Team B', 'Location', 'Gender', 'League/Event', 'Priority', 'Status']
print(df_sorted[output_columns])

























































































    Match No.       Date Time (IST)                 Team A            Team B  \
0          37 2024-08-16   10:00:00           South Africa          Pakistan   
1          16 2024-08-16   14:00:00                  India          Pakistan   
2           2 2024-08-16   15:00:00                England         Australia   
3           8 2024-08-15   14:00:00                  India          Pakistan   
4           1 2024-08-15   14:00:00                  India          Pakistan   
5          51 2024-08-16   20:00:00  Kolkata Knight Riders  Rajasthan Royals   
6          58 2024-08-16   20:00:00   Lucknow Super Giants    Delhi Capitals   
7          79 2024-08-16   20:00:00           Punjab Kings    Mumbai Indians   
8          65 2024-08-16   16:00:00         Mumbai Indians    Delhi Capitals   
9          72 2024-08-16   19:00:00                  India         Australia   
10          9 2024-08-16   20:00: